In [28]:
import requests
import time
import re
import json
import os
import pymysql

In [3]:
from bs4 import BeautifulSoup


In [11]:
HEADERS = (
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.119 Safari/537.36',
    'Opera/9.80 (Windows NT 6.0) Presto/2.12.388 Version/12.14',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:33.0) Gecko/20120101 Firefox/33.0',
    'Mozilla/5.0 (MSIE 10.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A'
    )

In [4]:
def make_cookies(cookies_path):
    """
    函数功能：构造特定用户的cookies
    参数：保存cookies的txt文件目录
    返回：cookies的字典对象
    """
    cookies = dict()
    with open(cookies_path, 'r')as f:
        data = f.read()
    for line in data.split(';'):
        key, value = line.split('=', 1)  # 指定分割次数为1，否则会报错
        cookies[key] = value
    return cookies

In [5]:
def get_html(url, cookies=None, headers=None):
    """
    函数功能：向服务器请求特定url的页面
    参数：
        url：链接地址
        cookies：该网站的用户的cookies
        headers：构造的请求头
    返回：响应的文本内容，即HTML数据
    """
    # user-agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.45 Safari/537.36 Edg/79.0.309.30'
    global HEADERS
    if headers is None:
        # headers = random.choice(HEADERS)
        headers = HEADERS[0]
    try:
        r = requests.get(url, headers={'User-Agent': headers}, cookies=cookies)
        r.encoding = 'utf-8'
        return r.text
    except Exception as e:
        print(e)
        return None

In [6]:
url = 'https://movie.douban.com/top250?start=0&filter='

In [7]:
cookies_path = 'cookies.txt'

In [8]:
cookies = make_cookies(cookies_path)

In [9]:
cookies

{'bid': 'knMOJ3Nlpr8',
 ' douban-fav-remind': '1',
 ' ll': '"118281"',
 ' __utma': '223695111.1343588672.1574575638.1574575638.1574575638.1',
 ' __utmb': '223695111.0.10.1574575638',
 ' __utmc': '223695111',
 ' __utmz': '223695111.1574575638.1.1.utmcsr=cn.bing.com|utmccn=(referral)|utmcmd=referral|utmcct=/',
 ' ap_v': '0,6.0',
 ' _pk_ses.100001.4cf6': '*',
 ' __yadk_uid': 'cdoUVLTYmwcKjAqdJJW4YrTWMiPY9PS5',
 ' _vwo_uuid_v2': 'D1FD78E494E0A5E16D76D27C810167B70|6620d50f1d0f300ff917920e854b769f',
 ' dbcl2': '"170987053:4H84xudGN7Y"',
 ' ck': 'MEng',
 ' push_noty_num': '0',
 ' push_doumail_num': '0',
 ' _pk_id.100001.4cf6': '9356e3931560b08b.1574575652.1.1574575733.1574575652.'}

In [12]:
html = get_html(url, cookies, headers=None)

In [13]:
soup = BeautifulSoup(html, 'html.parser')

In [14]:
soup.title.string

'\n豆瓣电影 Top 250\n'

In [15]:
div_list = soup.find_all('div', 'hd')

In [16]:
div_list

[<div class="hd">
 <a class="" href="https://movie.douban.com/subject/1292052/">
 <span class="title">肖申克的救赎</span>
 <span class="title"> / The Shawshank Redemption</span>
 <span class="other"> / 月黑高飞(港)  /  刺激1995(台)</span>
 </a>
 <span class="playable">[可播放]</span>
 </div>, <div class="hd">
 <a class="" href="https://movie.douban.com/subject/1291546/">
 <span class="title">霸王别姬</span>
 <span class="other"> / 再见，我的妾  /  Farewell My Concubine</span>
 </a>
 <span class="playable">[可播放]</span>
 </div>, <div class="hd">
 <a class="" href="https://movie.douban.com/subject/1292720/">
 <span class="title">阿甘正传</span>
 <span class="title"> / Forrest Gump</span>
 <span class="other"> / 福雷斯特·冈普</span>
 </a>
 <span class="playable">[可播放]</span>
 </div>, <div class="hd">
 <a class="" href="https://movie.douban.com/subject/1295644/">
 <span class="title">这个杀手不太冷</span>
 <span class="title"> / Léon</span>
 <span class="other"> / 杀手莱昂  /  终极追杀令(台)</span>
 </a>
 <span class="playable">[可播放]</span>
 <

In [23]:
movies = {}
for div in div_list:
    href = div.a['href']
    title = div.span.text
    movies[title] = href

In [17]:
d = div_list[0]

In [18]:
h = d.a['href']

In [20]:
s = d.span

In [22]:
s.text


'肖申克的救赎'

In [24]:
movies

{'肖申克的救赎': 'https://movie.douban.com/subject/1292052/',
 '霸王别姬': 'https://movie.douban.com/subject/1291546/',
 '阿甘正传': 'https://movie.douban.com/subject/1292720/',
 '这个杀手不太冷': 'https://movie.douban.com/subject/1295644/',
 '美丽人生': 'https://movie.douban.com/subject/1292063/',
 '泰坦尼克号': 'https://movie.douban.com/subject/1292722/',
 '千与千寻': 'https://movie.douban.com/subject/1291561/',
 '辛德勒的名单': 'https://movie.douban.com/subject/1295124/',
 '盗梦空间': 'https://movie.douban.com/subject/3541415/',
 '忠犬八公的故事': 'https://movie.douban.com/subject/3011091/',
 '机器人总动员': 'https://movie.douban.com/subject/2131459/',
 '海上钢琴师': 'https://movie.douban.com/subject/1292001/',
 '三傻大闹宝莱坞': 'https://movie.douban.com/subject/3793023/',
 '放牛班的春天': 'https://movie.douban.com/subject/1291549/',
 '楚门的世界': 'https://movie.douban.com/subject/1292064/',
 '星际穿越': 'https://movie.douban.com/subject/1889243/',
 '大话西游之大圣娶亲': 'https://movie.douban.com/subject/1292213/',
 '龙猫': 'https://movie.douban.com/subject/1291560/',
 '熔炉'

In [26]:
def soup_parser(soup):
    div_list = soup.find_all('div', 'hd')
    global movies
    for div in div_list:
        href = div.a['href']
        title = div.span.text
        movies[title] = href
    

In [27]:
url = 'https://movie.douban.com/top250?start={}&filter='

In [29]:
def conn_sql():
    conn = pymysql.connect(host='localhost', port=3306, user='root', password='mysql', database='movie', charset='utf8')
    cursor = conn.cursor()
    return cursor
    

In [30]:
del conn_sql

In [31]:
conn = pymysql.connect(host='localhost', port=3306, user='root', password='mysql', database='movie', charset='utf8')
cursor = conn.cursor()

In [32]:
sql = 'select * from movie_basic;'
row_count = cursor.execute(sql)

In [33]:
row_count

0

In [38]:
for key, value in movies.items():
    try:
        sql = 'insert into movie_basic value(0, "{}", "{}");'.format(key, value)
        print(sql)
        row_count = cursor.execute(sql)
        print(row_count)
        conn.commit()
    except Exception as e:
        print(e)
        conn.rollback()


insert into movie_basic value(0, "肖申克的救赎", "https://movie.douban.com/subject/1292052/");
1
insert into movie_basic value(0, "霸王别姬", "https://movie.douban.com/subject/1291546/");
1
insert into movie_basic value(0, "阿甘正传", "https://movie.douban.com/subject/1292720/");
1
insert into movie_basic value(0, "这个杀手不太冷", "https://movie.douban.com/subject/1295644/");
1
insert into movie_basic value(0, "美丽人生", "https://movie.douban.com/subject/1292063/");
1
insert into movie_basic value(0, "泰坦尼克号", "https://movie.douban.com/subject/1292722/");
1
insert into movie_basic value(0, "千与千寻", "https://movie.douban.com/subject/1291561/");
1
insert into movie_basic value(0, "辛德勒的名单", "https://movie.douban.com/subject/1295124/");
1
insert into movie_basic value(0, "盗梦空间", "https://movie.douban.com/subject/3541415/");
1
insert into movie_basic value(0, "忠犬八公的故事", "https://movie.douban.com/subject/3011091/");
1
insert into movie_basic value(0, "机器人总动员", "https://movie.douban.com/subject/2131459/");
1
insert i

In [40]:
sql = 'select * from movie_basic;'
row_count = cursor.execute(sql)

In [45]:
row_count.text

AttributeError: 'int' object has no attribute 'text'

In [46]:
url


'https://movie.douban.com/top250?start={}&filter='

In [57]:
import random
url = 'https://movie.douban.com/top250?start={}&filter='
movies = {}
for i in range(0, 10):
    url2 = url.format(i * 25)
    print(url2)
    html = get_html(url2, cookies, headers=None)
    try:
        soup = BeautifulSoup(html, 'html.parser')
        soup_parser(soup)
    except Exception as e:
        print(e)
    time.sleep(random.randint(3, 5))

https://movie.douban.com/top250?start=0&filter=
https://movie.douban.com/top250?start=25&filter=
https://movie.douban.com/top250?start=50&filter=
https://movie.douban.com/top250?start=75&filter=
https://movie.douban.com/top250?start=100&filter=
https://movie.douban.com/top250?start=125&filter=
https://movie.douban.com/top250?start=150&filter=
https://movie.douban.com/top250?start=175&filter=
https://movie.douban.com/top250?start=200&filter=
https://movie.douban.com/top250?start=225&filter=


In [58]:
movies

{'肖申克的救赎': 'https://movie.douban.com/subject/1292052/',
 '霸王别姬': 'https://movie.douban.com/subject/1291546/',
 '阿甘正传': 'https://movie.douban.com/subject/1292720/',
 '这个杀手不太冷': 'https://movie.douban.com/subject/1295644/',
 '美丽人生': 'https://movie.douban.com/subject/1292063/',
 '泰坦尼克号': 'https://movie.douban.com/subject/1292722/',
 '千与千寻': 'https://movie.douban.com/subject/1291561/',
 '辛德勒的名单': 'https://movie.douban.com/subject/1295124/',
 '盗梦空间': 'https://movie.douban.com/subject/3541415/',
 '忠犬八公的故事': 'https://movie.douban.com/subject/3011091/',
 '机器人总动员': 'https://movie.douban.com/subject/2131459/',
 '海上钢琴师': 'https://movie.douban.com/subject/1292001/',
 '三傻大闹宝莱坞': 'https://movie.douban.com/subject/3793023/',
 '放牛班的春天': 'https://movie.douban.com/subject/1291549/',
 '楚门的世界': 'https://movie.douban.com/subject/1292064/',
 '星际穿越': 'https://movie.douban.com/subject/1889243/',
 '大话西游之大圣娶亲': 'https://movie.douban.com/subject/1292213/',
 '龙猫': 'https://movie.douban.com/subject/1291560/',
 '熔炉'

In [59]:
for key, value in movies.items():
    try:
        sql = 'insert into movie_basic value(0, "{}", "{}");'.format(key, value)
        print(sql)
        row_count = cursor.execute(sql)
        print(row_count)
        conn.commit()
    except Exception as e:
        print(e)
        conn.rollback()

insert into movie_basic value(0, "肖申克的救赎", "https://movie.douban.com/subject/1292052/");
1
insert into movie_basic value(0, "霸王别姬", "https://movie.douban.com/subject/1291546/");
1
insert into movie_basic value(0, "阿甘正传", "https://movie.douban.com/subject/1292720/");
1
insert into movie_basic value(0, "这个杀手不太冷", "https://movie.douban.com/subject/1295644/");
1
insert into movie_basic value(0, "美丽人生", "https://movie.douban.com/subject/1292063/");
1
insert into movie_basic value(0, "泰坦尼克号", "https://movie.douban.com/subject/1292722/");
1
insert into movie_basic value(0, "千与千寻", "https://movie.douban.com/subject/1291561/");
1
insert into movie_basic value(0, "辛德勒的名单", "https://movie.douban.com/subject/1295124/");
1
insert into movie_basic value(0, "盗梦空间", "https://movie.douban.com/subject/3541415/");
1
insert into movie_basic value(0, "忠犬八公的故事", "https://movie.douban.com/subject/3011091/");
1
insert into movie_basic value(0, "机器人总动员", "https://movie.douban.com/subject/2131459/");
1
insert i

1
insert into movie_basic value(0, "哈利·波特与密室", "https://movie.douban.com/subject/1296996/");
1
insert into movie_basic value(0, "东京物语", "https://movie.douban.com/subject/1291568/");
1
insert into movie_basic value(0, "撞车", "https://movie.douban.com/subject/1388216/");
1
insert into movie_basic value(0, "聚焦", "https://movie.douban.com/subject/25954475/");
1
insert into movie_basic value(0, "遗愿清单", "https://movie.douban.com/subject/1867345/");
1
insert into movie_basic value(0, "发条橙", "https://movie.douban.com/subject/1292233/");
1
insert into movie_basic value(0, "梦之安魂曲", "https://movie.douban.com/subject/1292270/");
1
insert into movie_basic value(0, "黑鹰坠落", "https://movie.douban.com/subject/1291824/");
1
insert into movie_basic value(0, "色，戒", "https://movie.douban.com/subject/1828115/");
1
insert into movie_basic value(0, "小偷家族", "https://movie.douban.com/subject/27622447/");
1
insert into movie_basic value(0, "E.T. 外星人", "https://movie.douban.com/subject/1294638/");
1
insert into mo

# 爬取影片详情的内容

In [60]:
url = 'https://movie.douban.com/subject/1292052/'

In [61]:
html = get_html(url, cookies, headers=None)

In [62]:
soup = BeautifulSoup(html, 'html.parser')

In [63]:
soup.title.text

'\n        肖申克的救赎 (豆瓣)\n'